## 1.0 Data
Data for this project contains simulated timestamps of user logins in a particular geographic location.

### 1.0.1 Task 1
The first task is to aggregate these login counts based on 15-minute intervals

### 1.0.1 Task 2
The second task is to visualize and describe the resulting time series of login counts in ways that best characterize the underlying patterns of the demand.

### 1.0.3 Task 3
The third task is to report/illustrate important features of the demand such as daily cycles
and to report data quality issues.